Building the model described in Faizi et. al. 2018 "A model of optimal protein allocation during phototrophic growth" Biosystems, 166: 26-36.

In [ ]:
import tellurium as te
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model Faizi
    
    var Ci_ext; # extracellular inorganic carbon
    var Ci_int; # intracellular inorganic carbon
    var Energy; # energy units (here representing the ATP, NADH, etc pool)
    var AminoAcid; # amino acids
    var Photoenzyme; # photosynthetic machinery
    var Transporter; # transporter protein for inorganic carbon
    var Metaenzyme; # metabolic multi-enzyme complex
    var Ribosome; # ribosome
    
    compartment Environment, Cell
    
    Environment = 1.0;
    Cell = 1.0;
    
    species Ci_ext, Ci_int, Energy, AminoAcid, Ribosome, Transporter, Metaenzyme, Photoenzyme;
    
    Ci_ext in Environment;
    Ci_int in Cell;
    Energy in Cell;
    AminoAcid in Cell;
    Ribosome in Cell; 
    Transporter in Cell; 
    Metaenzyme in Cell; 
    Photoenzyme in Cell; 
    
    // Initializing variables
    Energy = 1.0e4
    AminoAcid = 1.0e4
    Ribosome = 1.0e4
    Transporter = 1.0e4
    Metaenzyme = 1.0e4
    Photoenzyme = 1.0e4 
    
    // Setting proportions of protein translation rates (beta_i > 0; sum(beta_i)<=1)    
    beta_R = 0.05
    beta_T = 0.25
    beta_M = 0.25
    beta_P = 0.025
 
    // Geometric Constants
    Perm_membrane = 0.0018; # permeability, dm min-1
    area_cell = 3.85e-9; # cell surface area, dm2 cell-1
    volume_cell = 2.24e-14; # L, cell-1
    N_a = 6.022e23; # Avogadro constant, mol-1
    
    // Kinetic Constants
    kcat_T = 726; # maximal import rate, min-1
    Km_T = 15; # Menten constant of T, uM
    kcat_M = 545; # maximal metabolic rate, min-1
    Km_M = 2441560; # Menten constant of M, molecules cell-1
    gamma_max = 1320; # maximal translation rate, aa min-1 molecules-1
    Km_AA = 10000; # Menten constant for aa, molecule cell-1
    Km_E = 10000; # Menten constant for e, molecule cell-1
    abs_xsec = 0.166; # effective absorption cross-section of P, m2 uE-1
    turnover_P = 1900; # maximal turnover rate of P, min-1
    k_d = 1.6e-6; # rate constant for photodamage
    K_i = 1.0e8; # product-inhibition constant for e synthesis, molecule cell-1

    // Material Constants 
    Dens_C = 1.4e10; # average cell density, aa cell-1
    n_R = 7358; # ribosome length, aa molecule-1
    n_T = 1681; # transporter length, aa molecule-1
    n_M = 28630; # metabolic enzyme complex length, aa molecule-1
    n_P = 95451; # photosynthetic enzyme complex length, aa molecule-1
    m_mu = 45; # e needed to create one aa
    m_gamma = 3; # e needed for one translational elongation step
    m_omega = 8; # e produced by Photosynthetic cycle
    m_c = 5; # average carbon chain length of aa
    
    // Assignment Rules
    v_t := kcat_T * Transporter * Ci_ext / (Km_T + Ci_ext) * Energy / (Km_E + Energy);
    
    gamma_R := beta_R * gamma_max / n_R * Ribosome * Energy / (Km_E + Energy) * AminoAcid / (Km_AA + AminoAcid) 
    gamma_T := beta_T * gamma_max / n_T * Ribosome * Energy / (Km_E + Energy) * AminoAcid / (Km_AA + AminoAcid) 
    gamma_M := beta_M * gamma_max / n_M * Ribosome * Energy / (Km_E + Energy) * AminoAcid / (Km_AA + AminoAcid)
    gamma_P := beta_P * gamma_max / n_P * Ribosome * Energy / (Km_E + Energy) * AminoAcid / (Km_AA + AminoAcid) 

    mu := v_t / m_c * 1. / Dens_c 
    
    // Rate Laws

    R_P: -> Ribosome; gamma_R
    
    R_C: Ribosome ->; mu * Ribosome
    
    T_P: -> Transporter; gamma_T
    
    T_C: Transporter ->; mu * Transporter

    M_P: -> Metaenzyme; gamma_M
    
    M_C: Metaenzyme ->; mu * Metaenzyme

    P_P: -> Photoenzyme; gamma_P
    
    P_C: Photoenzyme ->; mu * Photoenzyme
   
end

In [ ]:
Faizi.reset()
Faizi.simulate(start = 0, end = 5.64e14, steps = 1000,selections =['time',])
Faizi.plot()

Faizi.steadyStateSelections = ['']
values = Faizi.getSteadyStateValues()
selections = Faizi.steadyStateSelections
print(values)
print(selections)